In [110]:
import numpy as np
import matplotlib.pyplot as plt
import skimage.io
from skimage.segmentation import slic
import time

import os

import tensorflow as tf
from tensorflow import keras 
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras import layers
from tensorflow.keras.layers import Layer
from tensorflow.keras import utils

from tensorflow.keras.utils import plot_model

import tensorflow.keras.backend as K

import sys
import importlib
#custom packages
function_dir = os.path.join(os.pardir, 'Functions')
sys.path.append(function_dir)

import segmentation as seg
import DLtools as ctools
importlib.reload(seg) #to make sure the last version of seg is used
importlib.reload(ctools)

<module 'DLtools' from '..\\Functions\\DLtools.py'>

In [3]:
sf = 4

In [5]:
#features extractor v2, we use the 3 first blocks of resnet pretrained on imagenet:
Inception = tf.keras.applications.InceptionResNetV2(
                                                    include_top=False,
                                                    weights="imagenet",
                                                    input_shape=(2464//sf,3280//sf,3),
                                                    pooling=None
                                                )


In [7]:
Inception.summary()
#plot_model(Inception, show_shapes=True, show_layer_names=True)

Model: "inception_resnet_v2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 616, 820, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 307, 409, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 307, 409, 32) 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 307, 409, 32) 0           batch_normalization[0][0]        
________________________________________________________________________________

In [17]:
feature_ex_inc = Model(inputs=Inception.input, outputs=[
                                            Inception.get_layer('block35_3').output]
                    ,name='features_ex',trainable=False)
feature_ex_inc.save_weights('Model_weights/features_extraction_pretrained_from_InceptionResNetv2.h5')
feature_ex_inc.summary()

Model: "features_ex"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 616, 820, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 307, 409, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma multiple             96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         multiple             0           batch_normalization[0][0]        
________________________________________________________________________________________

In [19]:
max_clusters = 10

In [153]:
#input image:
img = layers.Input((2464//sf,3280//sf,3),name = 'img')
#input superpixel: 

#take the feature extractor:

#features = feature_ex_ref(img)
features = feature_ex_inc(img)         

#get the initial size from the image

### for VGG19:
#fullscale = layers.UpSampling2D(name='revmaxpool')(features)
#combine the channels:
#cluster = layers.Conv2D(max_clusters,1,name = 'combiner')(fullscale)

###for inceptionResNetv2:
strides = [8,8]#[img.shape[1]//(features.shape[1]-1),(img.shape[2])//(features.shape[2]-1)]
pad = [imgd-stride*(featuresd) for imgd,stride,featuresd in zip(img.shape[1:3],strides,features.shape[1:3])]
kernel_size = [7,7]
cluster = layers.Conv2DTranspose(max_clusters,kernel_size,strides=strides, name='combiner',padding='valid')(features)
padded_cluster = ctools.ReflectionPadding2D(padding =[pad[0]//2,pad[1]//2])(cluster)

clustern = layers.LayerNormalization(name = 'cluster_normalization',axis = (1,2),
                                    scale = False,center = False)(padded_cluster)

#get the clustered image:
#constrain the superpixels:

unsupervised_seg = Model(inputs = img,outputs=clustern)


In [154]:
unsupervised_seg.summary()

Model: "model_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
img (InputLayer)             [(None, 616, 820, 3)]     0         
_________________________________________________________________
features_ex (Functional)     (None, 74, 100, 320)      812144    
_________________________________________________________________
combiner (Conv2DTranspose)   (None, 592, 800, 10)      156810    
_________________________________________________________________
reflection_padding2d_38 (Ref (None, 616, 820, 10)      0         
_________________________________________________________________
cluster_normalization (Layer (None, 616, 820, 10)      0         
Total params: 968,954
Trainable params: 156,810
Non-trainable params: 812,144
_________________________________________________________________


In [132]:
cluster

<KerasTensor: shape=(None, 592, 800, 10) dtype=float32 (created by layer 'combiner')>

In [86]:
pad = [imgd-stride*featuresd for imgd,stride,featuresd in zip(img.shape[1:3],strides,features.shape[1:3])]


In [115]:
ref = ctools.ReflectionPadding2D(padding =[pad[0]//2,pad[1]//2])

In [116]:
f = layers.Conv2DTranspose(max_clusters,kernel_size,strides=strides, name='combiner')(features)
padded = ref(f)


In [117]:
padded

<KerasTensor: shape=(None, 616, 820, 10) dtype=float32 (created by layer 'reflection_padding2d_21')>

In [95]:
features

<KerasTensor: shape=(None, 74, 100, 320) dtype=float32 (created by layer 'features_ex')>

In [109]:
pad[0]

24